In [1]:
import pandas as pd
pd.options.display.max_rows = 999

# load data

In [2]:
address_MBL = pd.read_csv('../data/address_MBL.csv', index_col = 0)

parking_permits = pd.read_csv('../data/City_of_Somerville_Parking_Permits.csv')

In [3]:
address_MBL.head()

,MBL,SITE_ADDR
0,1-C-5,"190 BOSTON AVE, Somerville, MA"
1,1-C-6,"188 BOSTON AVE, Somerville, MA"
2,1-C-7,"184 BOSTON AVE, Somerville, MA"
3,1-C-8,"180 BOSTON AVE, Somerville, MA"
4,1-C-9,"178 BOSTON AVE, Somerville, MA"


In [4]:
parking_permits.head()

,type_code,type_name,issued,effective,expiration,st_addr,unit_num,city,state,zip_code
0,WD,Moving Van,02/23/2017 12:00:00 AM,03/01/2017 12:00:00 AM,03/01/2017 12:00:00 AM,69 ADAMS ST,1,SOMERVILLE,MA,2145.0
1,G,Visitor,05/22/2017 12:00:00 AM,04/01/2017 12:00:00 AM,04/30/2018 12:00:00 AM,37 SEWALL ST,,SOMERVILLE,MA,2145.0
2,G,Visitor,05/22/2017 12:00:00 AM,04/01/2017 12:00:00 AM,04/30/2018 12:00:00 AM,37 SEWALL ST,,SOMERVILLE,MA,2145.0
3,G,Visitor,07/07/2017 12:00:00 AM,07/06/2017 12:00:00 AM,06/30/2018 12:00:00 AM,25 BEACON ST,5,SOMERVILLE,MA,2143.0
4,G,Visitor,07/07/2017 12:00:00 AM,07/06/2017 12:00:00 AM,06/30/2018 12:00:00 AM,25 BEACON ST,5,SOMERVILLE,MA,2143.0


## Clean parking permits

In [5]:
is_residential = (parking_permits.type_name == 'Residential ') | (parking_permits.type_name == 'Residential')
parking_permits = parking_permits[is_residential]

parking_permits['issued'] = pd.to_datetime(parking_permits['issued'])
was_issued_in_2018 = parking_permits.issued.dt.year == 2018
parking_permits = parking_permits[was_issued_in_2018]


In [23]:
parking_permits_counts = parking_permits.groupby('st_addr').size().reset_index()
parking_permits_counts.columns = ['st_addr','PARKING_PERMIT_COUNT']
parking_permits_counts.head()

,st_addr,PARKING_PERMIT_COUNT
0,1 ALDERSEY ST,1
1,1 AVON ST,3
2,1 BEACON ST,1
3,1 BELMONT SQ,3
4,1 BENTON RD,3


In [28]:
parking_permits_counts.st_addr = parking_permits_counts.st_addr.apply(lambda x: x.replace(' AV', ' AVE'))

## Merge with MBL

In [29]:
address_MBL['st_addr'] = address_MBL.SITE_ADDR.apply(lambda x: x.split(',')[0]).replace(' AV', ' AVE')

In [35]:
address_MBL = address_MBL.merge(parking_permits_counts, how = 'left').fillna(0)

In [42]:
address_MBL[['MBL','PARKING_PERMIT_COUNT']].to_csv('../data/parking_permit_counts.csv')